In [ ]:
from TransversalitySheaf import *
from ConleySheaf import *
from FlowSheaf import *
from pychomp import *

In [ ]:
#   mapping = {}
#   #Step 1. Construct the complex for phase space
#   bounds,boxes = phase_specs
#   geometric_phase_complex = GeometricCubicalComplex(bounds,boxes)
#   phase_complex = geometric_phase_complex.complex
#   #Step 2. For each cell, compute transversality complex, followed by graded_cell complex
#   # 1 . The transversality complex, returns (phase_complex, discrete_flow)
#   B = base_complex.complex
#   for base_vertex in B(0):
#     params = [coord[0] for coord in base_complex.geometry(base_vertex)]
#     (C,discrete_flow) = TransversalityComplex(geometric_phase_complex, ODE_model, params, num_samples)
#     mapping[base_vertex] = FlowComplex(C,discrete_flow)
#     #sheaf_mapping[base_vertex] = FlowGradedComplex(*TransversalityComplex(geometric_phase_complex, ODE_model, params, num_samples))
#   return CubicalSheaf(base_complex, lambda x: mapping[x]) 

In [ ]:
phase_boxes = [3]
phase_bounds = [[0,3]]
phase_complex = GeometricCubicalComplex(phase_bounds,phase_boxes)

In [ ]:
X = phase_complex.complex

In [ ]:
for cube in X:
    print(cube,X.cell_dim(cube),phase_complex.geometry(cube))

In [ ]:
param_boxes = [1]
param_bounds = [[0,1]]
param_complex = GeometricCubicalComplex(param_bounds,param_boxes)
B = param_complex.complex

In [ ]:
for cube in B:
        print(cube,B.cell_dim(cube),param_complex.geometry(cube))

In [ ]:
F0 = {4: set(), 5: {4}, 6: {5}, 7: {6,4}}
F1 = {4: set(), 5: {4,6}, 6: set(), 7: {6,4}}

In [ ]:
FC0 = FlowComplex(X, F0)
FC1 = FlowComplex(X, F1)

In [ ]:
vertex_flow_sheaf_mapping = {0: FC0, 1: FC1}

In [ ]:
vertex_flow_sheaf = CubicalSheaf(param_complex, vertex_flow_sheaf_mapping)

In [ ]:
flow_sheaf = SheafFromVertices(vertex_flow_sheaf)

In [ ]:
FC2 = flow_sheaf.mapping[2]

In [ ]:
F2 = FC2.discrete_flow

In [ ]:
for top_cell in [4,5,6,7]:
    print(F2[top_cell])

In [ ]:
graded_sheaf = GradeFlowSheaf(flow_sheaf)

In [ ]:
GC0 = graded_sheaf.mapping[0]
GC1 = graded_sheaf.mapping[1]

In [ ]:
DrawGradedComplex(GC0.graded_complex, GC0.poset)

In [ ]:
DrawGradedComplex(GC1.graded_complex,GC1.poset)

In [ ]:
GC2 = graded_sheaf.mapping[2]

In [ ]:
DrawGradedComplex(GC2.graded_complex,GC2.poset)

In [ ]:
#Check poset morphism
base_poset = param_complex.face_poset

In [ ]:
base_poset.edges()

In [ ]:
GCMap = graded_sheaf.mapping[(2,0)]

In [ ]:
poset_map = GCMap.poset_map

In [ ]:
for v in [0,1,2,3]:
    print(v,poset_map(v))

In [ ]:
def poset_iso(source,target,gcmap):
    poset_map = gcmap.poset_map
    img = [poset_map(v) for v in source.poset.vertices()]
    print(img)
    img_set = set(img)
    print(img_set)
    #Check injective
    if not len(img)==len(img_set):
        return False
    #Check surjective
    if not img_set==target.poset.vertices():
        return False
    return True

In [ ]:
GCMap.poset_iso(GC0,GC2)

In [ ]:
GCMap = graded_sheaf.mapping[(2,1)]
GCMap.poset_iso(GC1,GC2)

In [ ]:
conley_sheaf = ConleySheaf(graded_sheaf)

In [ ]:
CC0 = conley_sheaf.mapping[0]
CC1 = conley_sheaf.mapping[1]
CC2 = conley_sheaf.mapping[2]

In [ ]:
#Check Conley complexes
DrawGradedComplex(CC0.graded_complex,CC0.poset)

In [ ]:
DrawGradedComplex(CC1.graded_complex,CC1.poset)

In [ ]:
DrawGradedComplex(CC2.graded_complex,CC2.poset)

In [ ]:
#Check transition matrices
face_poset = conley_sheaf.base_complex.face_poset

In [ ]:
face_poset.edges()

In [ ]:
conley_map = conley_sheaf.mapping[(2,1)]
transition_matrix = conley_map.chain_map
t_inverse = conley_map.homotopy_inv
for cell in CC2.graded_complex.complex():
    print(cell,t_inverse({cell}))

In [ ]:
conley_complex = CC1.graded_complex.complex()
d = conley_complex.dimension()
for i in range(0,d+1):
    print("Cells of dimension "+str(i))
    for cell in conley_complex(i):
        print(cell)

In [ ]:
poset1,GC1 = FlowGradedComplex(X, lambda x : F1[x])
poset2,GC2 = FlowGradedComplex(X, lambda x : F2[x])

In [ ]:
tower1 = ConnectionMatrixTower(GC1)
tower2 = ConnectionMatrixTower(GC2)

In [ ]:
tower1

In [ ]:
tower2

In [ ]:
def lower ( chain, tower ):
    for m in tower[1:]:
        chain = m.complex().lower(chain)
    return chain
def lift ( chain, tower ):
    for m in reversed(tower[1:]):
        chain = m.complex().lift(chain)
    return chain

In [ ]:
chain_map = lambda x : lower(lift(x, tower1), tower2)
homotopy_inverse = lambda x : lower(lift(x, tower2), tower1)

In [ ]:
lift({0},tower1)

In [ ]:
lift({0},tower2)

In [ ]:
lower({1},tower1)

In [ ]:
conley_complex = CC1.graded_complex.complex()
d = conley_complex.dimension()
for i in range(0,d+1):
    print("Cells of dimension "+str(i))
    for cell in conley_complex(i):
        print(cell,chain_map({cell}))